# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (10, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 10 to 20:
the lime is her least liked fruit , but the banana is my least liked .
he saw a old yellow truck .
india is rainy during june , and it is sometimes warm in november .
that cat was my most loved animal .
he dislikes grapefruit , limes , and lemons .
her least liked fruit is the lemon , but his least liked is the grapefruit .
california is never cold during february , but it is sometimes freezing in june .
china is usually pleasant during autumn , and it is usually quiet in october .
paris is never freezing during november , but it is wonderful in october .
the united states is never rainy during january , but it is sometimes mild in october .

French sentences 10 to 20:
la chaux est son moins aimé des fruits , mais la banane est mon moins aimé.
il a vu un vieux camion jaune .
inde est pluvieux en juin , et il est parfois

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Just go through the text and transform it.
    source_id_text = []
    for idx, line in enumerate(source_text.split('\n')):
        source_id_text.append([])
        for word in line.split():
            source_id_text[idx].append(source_vocab_to_int[word])
        
    target_id_text = []
    for idx, line in enumerate(target_text.split('\n')):
        target_id_text.append([])
        for word in line.split():
            target_id_text[idx].append(target_vocab_to_int[word])
        target_id_text[idx].append(target_vocab_to_int['<EOS>'])

    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, shape=[None,None], name= "input")
    targets = tf.placeholder(tf.int32, shape=[None,None], name= "targets")
    lrate = tf.placeholder(tf.float32, name= "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    target_seq_lenth = tf.placeholder(tf.int32, shape=[None], name= "target_sequence_length")
    max_target_len = tf.reduce_max(target_seq_lenth, name= 'max_target_len')
    source_seq_length = tf.placeholder(tf.int32, shape=[None], name= "source_sequence_length")
    return (inputs, targets, lrate, keep_prob, target_seq_lenth, max_target_len, source_seq_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'F

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # Create a constant tensor with the 'go id'.
    go_id = tf.constant(target_vocab_to_int['<GO>'], shape=(batch_size,1), dtype=tf.int32)
    # Concatenate the vector without the last word id with the go ids vector
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Build the lstm cells wrapped in dropout
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    # Creathe embedding layer.
    embed_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    # If we don't have an initial zero state, provide a dtype.
    output, state = tf.nn.dynamic_rnn(stacked_lstm, embed_encoder, source_sequence_length, dtype=tf.float32)
    return (output, state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainig_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, trainig_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(basic_decoder,maximum_iterations=max_summary_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # Convert the start_ids to be a vector with batch size (the go id repeated batch size times)
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    # Create the embedding helper.
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, embedding_helper, encoder_state, output_layer)
    f_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,maximum_iterations=max_target_sequence_length)
    return f_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Use the same proess as in the encoding layer.
    def build_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm_drop
    # Stack them all
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
       tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob) 
    with tf.variable_scope("decode", reuse=True):
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    tr_decoder_output, inf_decoder_output = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return tr_decoder_output, inf_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 40
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 60
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 40
decoding_embedding_size = 40
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.0014, Validation Accuracy: 0.0004, Loss: 5.8576
Epoch   0 Batch    2/538 - Train Accuracy: 0.0969, Validation Accuracy: 0.1103, Loss: 5.8194
Epoch   0 Batch    3/538 - Train Accuracy: 0.2303, Validation Accuracy: 0.3175, Loss: 5.7839
Epoch   0 Batch    4/538 - Train Accuracy: 0.2393, Validation Accuracy: 0.3173, Loss: 5.7414
Epoch   0 Batch    5/538 - Train Accuracy: 0.2647, Validation Accuracy: 0.3176, Loss: 5.6832
Epoch   0 Batch    6/538 - Train Accuracy: 0.2712, Validation Accuracy: 0.3180, Loss: 5.6234
Epoch   0 Batch    7/538 - Train Accuracy: 0.2465, Validation Accuracy: 0.3178, Loss: 5.5782
Epoch   0 Batch    8/538 - Train Accuracy: 0.2473, Validation Accuracy: 0.3182, Loss: 5.5058
Epoch   0 Batch    9/538 - Train Accuracy: 0.2441, Validation Accuracy: 0.3182, Loss: 5.4378
Epoch   0 Batch   10/538 - Train Accuracy: 0.2225, Validation Accuracy: 0.3180, Loss: 5.3867
Epoch   0 Batch   11/538 - Train Accuracy: 0.2365, Validation Accuracy

Epoch   0 Batch   91/538 - Train Accuracy: 0.2748, Validation Accuracy: 0.3542, Loss: 3.2279
Epoch   0 Batch   92/538 - Train Accuracy: 0.2965, Validation Accuracy: 0.3542, Loss: 3.2048
Epoch   0 Batch   93/538 - Train Accuracy: 0.2727, Validation Accuracy: 0.3542, Loss: 3.2228
Epoch   0 Batch   94/538 - Train Accuracy: 0.2742, Validation Accuracy: 0.3548, Loss: 3.2553
Epoch   0 Batch   95/538 - Train Accuracy: 0.3420, Validation Accuracy: 0.3556, Loss: 2.9722
Epoch   0 Batch   96/538 - Train Accuracy: 0.3240, Validation Accuracy: 0.3556, Loss: 3.0368
Epoch   0 Batch   97/538 - Train Accuracy: 0.2812, Validation Accuracy: 0.3599, Loss: 3.1949
Epoch   0 Batch   98/538 - Train Accuracy: 0.3320, Validation Accuracy: 0.3601, Loss: 3.0184
Epoch   0 Batch   99/538 - Train Accuracy: 0.2963, Validation Accuracy: 0.3601, Loss: 3.1836
Epoch   0 Batch  100/538 - Train Accuracy: 0.2947, Validation Accuracy: 0.3629, Loss: 3.1640
Epoch   0 Batch  101/538 - Train Accuracy: 0.3016, Validation Accuracy

Epoch   0 Batch  181/538 - Train Accuracy: 0.3617, Validation Accuracy: 0.4355, Loss: 2.8909
Epoch   0 Batch  182/538 - Train Accuracy: 0.3615, Validation Accuracy: 0.4393, Loss: 2.8885
Epoch   0 Batch  183/538 - Train Accuracy: 0.4038, Validation Accuracy: 0.4370, Loss: 2.7258
Epoch   0 Batch  184/538 - Train Accuracy: 0.4044, Validation Accuracy: 0.4387, Loss: 2.7306
Epoch   0 Batch  185/538 - Train Accuracy: 0.3844, Validation Accuracy: 0.4448, Loss: 2.7969
Epoch   0 Batch  186/538 - Train Accuracy: 0.3994, Validation Accuracy: 0.4441, Loss: 2.7627
Epoch   0 Batch  187/538 - Train Accuracy: 0.4297, Validation Accuracy: 0.4442, Loss: 2.6235
Epoch   0 Batch  188/538 - Train Accuracy: 0.3730, Validation Accuracy: 0.4490, Loss: 2.8621
Epoch   0 Batch  189/538 - Train Accuracy: 0.3861, Validation Accuracy: 0.4462, Loss: 2.8205
Epoch   0 Batch  190/538 - Train Accuracy: 0.4146, Validation Accuracy: 0.4485, Loss: 2.7251
Epoch   0 Batch  191/538 - Train Accuracy: 0.4133, Validation Accuracy

Epoch   0 Batch  271/538 - Train Accuracy: 0.3957, Validation Accuracy: 0.4572, Loss: 2.6293
Epoch   0 Batch  272/538 - Train Accuracy: 0.3977, Validation Accuracy: 0.4693, Loss: 2.7000
Epoch   0 Batch  273/538 - Train Accuracy: 0.4246, Validation Accuracy: 0.4702, Loss: 2.5741
Epoch   0 Batch  274/538 - Train Accuracy: 0.3760, Validation Accuracy: 0.4503, Loss: 2.6875
Epoch   0 Batch  275/538 - Train Accuracy: 0.4100, Validation Accuracy: 0.4689, Loss: 2.6164
Epoch   0 Batch  276/538 - Train Accuracy: 0.4150, Validation Accuracy: 0.4707, Loss: 2.6029
Epoch   0 Batch  277/538 - Train Accuracy: 0.4072, Validation Accuracy: 0.4542, Loss: 2.5704
Epoch   0 Batch  278/538 - Train Accuracy: 0.4027, Validation Accuracy: 0.4544, Loss: 2.5529
Epoch   0 Batch  279/538 - Train Accuracy: 0.4160, Validation Accuracy: 0.4696, Loss: 2.5601
Epoch   0 Batch  280/538 - Train Accuracy: 0.4377, Validation Accuracy: 0.4689, Loss: 2.4647
Epoch   0 Batch  281/538 - Train Accuracy: 0.3980, Validation Accuracy

Epoch   0 Batch  361/538 - Train Accuracy: 0.4405, Validation Accuracy: 0.4700, Loss: 2.2876
Epoch   0 Batch  362/538 - Train Accuracy: 0.4462, Validation Accuracy: 0.4661, Loss: 2.2538
Epoch   0 Batch  363/538 - Train Accuracy: 0.4332, Validation Accuracy: 0.4467, Loss: 2.2311
Epoch   0 Batch  364/538 - Train Accuracy: 0.3922, Validation Accuracy: 0.4506, Loss: 2.3685
Epoch   0 Batch  365/538 - Train Accuracy: 0.4252, Validation Accuracy: 0.4602, Loss: 2.2989
Epoch   0 Batch  366/538 - Train Accuracy: 0.4127, Validation Accuracy: 0.4599, Loss: 2.3406
Epoch   0 Batch  367/538 - Train Accuracy: 0.3902, Validation Accuracy: 0.4480, Loss: 2.3492
Epoch   0 Batch  368/538 - Train Accuracy: 0.4935, Validation Accuracy: 0.4512, Loss: 2.0157
Epoch   0 Batch  369/538 - Train Accuracy: 0.4219, Validation Accuracy: 0.4641, Loss: 2.3280
Epoch   0 Batch  370/538 - Train Accuracy: 0.4043, Validation Accuracy: 0.4663, Loss: 2.3682
Epoch   0 Batch  371/538 - Train Accuracy: 0.4408, Validation Accuracy

...

Epoch  39 Batch  157/538 - Train Accuracy: 0.9661, Validation Accuracy: 0.9483, Loss: 0.0488
Epoch  39 Batch  158/538 - Train Accuracy: 0.9521, Validation Accuracy: 0.9455, Loss: 0.0554
Epoch  39 Batch  159/538 - Train Accuracy: 0.9354, Validation Accuracy: 0.9505, Loss: 0.0648
Epoch  39 Batch  160/538 - Train Accuracy: 0.9355, Validation Accuracy: 0.9506, Loss: 0.0569
Epoch  39 Batch  161/538 - Train Accuracy: 0.9443, Validation Accuracy: 0.9478, Loss: 0.0601
Epoch  39 Batch  162/538 - Train Accuracy: 0.9490, Validation Accuracy: 0.9508, Loss: 0.0587
Epoch  39 Batch  163/538 - Train Accuracy: 0.9403, Validation Accuracy: 0.9558, Loss: 0.0669
Epoch  39 Batch  164/538 - Train Accuracy: 0.9551, Validation Accuracy: 0.9512, Loss: 0.0684
Epoch  39 Batch  165/538 - Train Accuracy: 0.9310, Validation Accuracy: 0.9444, Loss: 0.0447
Epoch  39 Batch  166/538 - Train Accuracy: 0.9633, Validation Accuracy: 0.9441, Loss: 0.0498
Epoch  39 Batch  167/538 - Train Accuracy: 0.9369, Validation Accuracy

Epoch  39 Batch  247/538 - Train Accuracy: 0.9545, Validation Accuracy: 0.9489, Loss: 0.0558
Epoch  39 Batch  248/538 - Train Accuracy: 0.9543, Validation Accuracy: 0.9458, Loss: 0.0585
Epoch  39 Batch  249/538 - Train Accuracy: 0.9472, Validation Accuracy: 0.9478, Loss: 0.0529
Epoch  39 Batch  250/538 - Train Accuracy: 0.9549, Validation Accuracy: 0.9482, Loss: 0.0530
Epoch  39 Batch  251/538 - Train Accuracy: 0.9508, Validation Accuracy: 0.9480, Loss: 0.0531
Epoch  39 Batch  252/538 - Train Accuracy: 0.9453, Validation Accuracy: 0.9544, Loss: 0.0470
Epoch  39 Batch  253/538 - Train Accuracy: 0.9420, Validation Accuracy: 0.9496, Loss: 0.0476
Epoch  39 Batch  254/538 - Train Accuracy: 0.9447, Validation Accuracy: 0.9439, Loss: 0.0642
Epoch  39 Batch  255/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9485, Loss: 0.0528
Epoch  39 Batch  256/538 - Train Accuracy: 0.9406, Validation Accuracy: 0.9490, Loss: 0.0526
Epoch  39 Batch  257/538 - Train Accuracy: 0.9611, Validation Accuracy

Epoch  39 Batch  337/538 - Train Accuracy: 0.9418, Validation Accuracy: 0.9498, Loss: 0.0538
Epoch  39 Batch  338/538 - Train Accuracy: 0.9578, Validation Accuracy: 0.9490, Loss: 0.0609
Epoch  39 Batch  339/538 - Train Accuracy: 0.9468, Validation Accuracy: 0.9453, Loss: 0.0506
Epoch  39 Batch  340/538 - Train Accuracy: 0.9383, Validation Accuracy: 0.9464, Loss: 0.0557
Epoch  39 Batch  341/538 - Train Accuracy: 0.9564, Validation Accuracy: 0.9469, Loss: 0.0543
Epoch  39 Batch  342/538 - Train Accuracy: 0.9548, Validation Accuracy: 0.9469, Loss: 0.0592
Epoch  39 Batch  343/538 - Train Accuracy: 0.9604, Validation Accuracy: 0.9460, Loss: 0.0562
Epoch  39 Batch  344/538 - Train Accuracy: 0.9619, Validation Accuracy: 0.9473, Loss: 0.0511
Epoch  39 Batch  345/538 - Train Accuracy: 0.9390, Validation Accuracy: 0.9471, Loss: 0.0577
Epoch  39 Batch  346/538 - Train Accuracy: 0.9276, Validation Accuracy: 0.9489, Loss: 0.0636
Epoch  39 Batch  347/538 - Train Accuracy: 0.9506, Validation Accuracy

Epoch  39 Batch  427/538 - Train Accuracy: 0.9268, Validation Accuracy: 0.9542, Loss: 0.0630
Epoch  39 Batch  428/538 - Train Accuracy: 0.9647, Validation Accuracy: 0.9522, Loss: 0.0477
Epoch  39 Batch  429/538 - Train Accuracy: 0.9598, Validation Accuracy: 0.9451, Loss: 0.0557
Epoch  39 Batch  430/538 - Train Accuracy: 0.9246, Validation Accuracy: 0.9451, Loss: 0.0514
Epoch  39 Batch  431/538 - Train Accuracy: 0.9369, Validation Accuracy: 0.9435, Loss: 0.0478
Epoch  39 Batch  432/538 - Train Accuracy: 0.9409, Validation Accuracy: 0.9457, Loss: 0.0659
Epoch  39 Batch  433/538 - Train Accuracy: 0.9563, Validation Accuracy: 0.9487, Loss: 0.0787
Epoch  39 Batch  434/538 - Train Accuracy: 0.9547, Validation Accuracy: 0.9537, Loss: 0.0573
Epoch  39 Batch  435/538 - Train Accuracy: 0.9373, Validation Accuracy: 0.9558, Loss: 0.0540
Epoch  39 Batch  436/538 - Train Accuracy: 0.9369, Validation Accuracy: 0.9556, Loss: 0.0588
Epoch  39 Batch  437/538 - Train Accuracy: 0.9676, Validation Accuracy

Epoch  39 Batch  517/538 - Train Accuracy: 0.9528, Validation Accuracy: 0.9540, Loss: 0.0545
Epoch  39 Batch  518/538 - Train Accuracy: 0.9561, Validation Accuracy: 0.9563, Loss: 0.0626
Epoch  39 Batch  519/538 - Train Accuracy: 0.9548, Validation Accuracy: 0.9553, Loss: 0.0542
Epoch  39 Batch  520/538 - Train Accuracy: 0.9547, Validation Accuracy: 0.9549, Loss: 0.0596
Epoch  39 Batch  521/538 - Train Accuracy: 0.9541, Validation Accuracy: 0.9526, Loss: 0.0639
Epoch  39 Batch  522/538 - Train Accuracy: 0.9563, Validation Accuracy: 0.9576, Loss: 0.0527
Epoch  39 Batch  523/538 - Train Accuracy: 0.9514, Validation Accuracy: 0.9513, Loss: 0.0589
Epoch  39 Batch  524/538 - Train Accuracy: 0.9475, Validation Accuracy: 0.9515, Loss: 0.0523
Epoch  39 Batch  525/538 - Train Accuracy: 0.9425, Validation Accuracy: 0.9505, Loss: 0.0603
Epoch  39 Batch  526/538 - Train Accuracy: 0.9509, Validation Accuracy: 0.9508, Loss: 0.0511
Epoch  39 Batch  527/538 - Train Accuracy: 0.9455, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [26]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    word_ids = []
    for word in sentence.lower().split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [31]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [37, 158, 219, 60, 11, 76, 164]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [65, 165, 248, 309, 66, 194, 307, 58, 1]
  French Words: il a de nouveau vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.